# Data Generator

This notebook generates all the data used in the paper. In general, every problem works as follows:

1. Pick entry from alphabet
2. Based on your delta, move to _another_ entry of the alphabet. Add the previous entry to a running string (tape).
3. If this entry is the end state (usually "stop"), return the tape.
4. Run a verifier to ensure the label is correct and (with 5% probability) otherwise flip it.

This is done for all deltas (`[0, 0.2, 0.45, 0.65, 0.85]`) and for about 5000 entries. We'll keep 2k for every test dataset, and 3k for training. 

Nuances on which alphabet, how to pick it, and how to move are problem-dependent. 

Some problems, like Hamiltonian, run for a long time because you need to first generate the graph and then verify it has a Hamiltonian path. 

## Shared functions/imports

In [2]:
from copy import deepcopy
import random
import json
from tqdm import tqdm

DELTAS = [0.2, 0.45, 0.65, 0.85]
MAX_DATA_LEN = 5000     # We'll generate 5k samples
TRAIN_LEN = 3000        # Out of which 3k will be train, 2k will be test
NOISE_THRESHOLD = 0.05  # 5% of the data is mislabelled

# Technically not a mislabel but a random noise injection
def mislabel_point():
    return random.uniform(0, 1) < NOISE_THRESHOLD


def verify_q(Q, P=None):
    # Verify whether your Q is:
    # (a) a well-formed probability matrix Q(\delta), and 
    # (b) (optionally) that in the case \delta = 0, Q(0) = P
    print("row, sum, any < 0")
    issues_found = False
    for d in DELTAS:
        print(d)
        for i, r in enumerate(Q(d)):
            print(i, sum(r), any([_r < 0 for _r in r]))
            if round(sum(r), 8) != 1.0 or any([_r < 0 for _r in r]):
                issues_found = True
        print("\n")
    print("Verifying P = Q(0)")
    if P is not None:
        for i, (r_p, r_q) in enumerate(zip(P, Q(0))):
            print(all([i == j for i,j in zip(r_p, r_q)]))
            if not all([i == j for i,j in zip(r_p, r_q)]):
                issues_found = True
    if issues_found:
        print("Check -- something needs fixing")
    else:
        print("All good!")


def get_label_balance(data):
    labels = {}
    for d in data:
        if d["Label"] not in labels:
            labels[d["Label"]] = 0
        labels[d["Label"]] += 1
    for k, v in labels.items():
        print(k, v*100./len(data))


def get_average_length(data):
    lens = sum([len(l["Entry"]) for l in data])
    print(lens/len(data))

## PARITY

The standard PARITY problem from CS: yes if there are an even number of zeros, no otherwise.

[Source](https://www.cs.montana.edu/webworks/projects/oldjunk/theory/contents/chapter002/section002/green/page002.html).

<img src="img/PARITY.jpg" alt="PARITY automaton" width="300px"/>

In [ ]:
def ecm(P, is_neg=False, max_length = None):
    """
    Automaton generating strings for the PARITY problem.
    Given a finite string of 0, 1 are even 0s?
    See https://arxiv.org/pdf/2202.09717.pdf for a description of this version of PARITY.
    """
    # P should be either P or Q (test)
    p00, p01, p0f = P[0]
    p10, p11, p1f = P[1]

    state = "S0"
    end_state = "S0" if is_neg else "S1"
    tape = "0" 
    steps = 0

    while True:
        x = random.randint(0, 1)
        #print(state, end_state, x, tape)
        if state == end_state:
            # Assume p0/1f is lower than the other two
            if state == "S0":
                if x <= p0f:
                    break
                else:
                    if x <= p00:
                        state = "S0"
                        tape += "1"
                    else:
                        state = "S1"
                        tape += "0"
            else:
                if x <= p1f:
                    break
                else:
                    if x <= p11:
                        state = "S1"
                        tape += "1"
                    else:
                        state = "S0"
                        tape += "0"
        else:
            # Only worry about probs
            if state == "S1":
                if x <= p10:
                    state = "S0"
                    tape += "0"
                else:
                    state = "S1"
                    tape += "1"
            else:
                if x <= p00:
                    state = "S0"
                    tape += "1"
                else:
                    state = "S1"
                    tape += "0"
        steps += 1
        if max_length is not None:
            if steps >= max_length:
                return None
    return tape


def tape_is_valid(tape, label, repeated, debug=False):
    if tape is None:
        if debug: print("Tape is none")
        return False
    if len(tape) < 4:
        if debug: print("Tape < 4")
        return False
    if tape in repeated:
        if debug: print("Tape is repeated")        
        return False
    zeros = tape.count("0")
    if label == 1 and zeros % 2 != 0:
        return False
    if label == 0 and zeros % 2 == 0:
        return False
    return True

In [ ]:
P = [[0.2, 0.8, 0], # can never hit S0
     [0.7, 0.2, 0.1]]

P_neg = [[0.7, 0.2, 0.1],
         [0.2, 0.8, 0]] # can never hit S1

corpus = []
repeated = {}

j = 0
pos, neg = 0, 0
pbar = tqdm(total = MAX_DATA_LEN)

while len(corpus) < MAX_DATA_LEN:
    # Positive
    tape = ecm(P)
    if not tape_is_valid(tape, 1, repeated):
        continue
    repeated[tape] = 1
    mislabel = mislabel_point()

    if pos < MAX_DATA_LEN//2:
        pos += 1
        corpus.append({"Entry": tape, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)

    # Negative
    tape = ecm(P_neg, is_neg=True)
    if not tape_is_valid(tape, 0, repeated):
        continue
    repeated[tape] = 1
    mislabel = mislabel_point()
    if neg < MAX_DATA_LEN//2:
        neg += 1
        corpus.append({"Entry": tape, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)


pbar.close()
random.shuffle(corpus)
train, test = corpus[:TRAIN_LEN], corpus[TRAIN_LEN:]

print(len(train), get_label_balance(train), get_average_length(train))
print("---")
print(len(test), get_label_balance(test), get_average_length(test))

with open("PARITY/corpus_parity_id_4k_new.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in train]
with open("PARITY/corpus_parity_id_4k_new_test.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in test]

In [ ]:
corpora_ood = {}
Q = lambda d: [[d, 1 - d, 0],
               [0.9 - d, d, 0.1]]
Q_neg = lambda d: [[0.9 - d, d, 0.1],
                   [d, 1 - d, 0]]

for d in tqdm(DELTAS):

    corpus = []
    repeated = {}
    j = 0
    pos, neg = 0, 0

    while len(corpus) < TRAIN_LEN: # We use train to have some slack
        # Positive
        tape = ecm(Q(d))
        if not tape_is_valid(tape, 1, repeated):
            continue
        repeated[tape] = 1
        mislabel = mislabel_point()
        if pos < TRAIN_LEN//2:
            pos += 1
            corpus.append({"Entry": tape, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
            j += 1
    
        # Negative
        tape = ecm(Q_neg(d), is_neg=True)
        if not tape_is_valid(tape, 0, repeated):
            continue
        repeated[tape] = 1
        mislabel = mislabel_point()
        if neg < TRAIN_LEN//2:
            neg += 1
            corpus.append({"Entry": tape, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
            j += 1

    random.shuffle(corpus)
    corpora_ood["delta_{}".format(d)] = corpus
    print(d, len(corpus), get_label_balance(corpus), get_average_length(corpus))
    
with open("PARITY/corpus_parity_ood_4k_test_new_with_negs.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(corpora_ood, ensure_ascii=False))

## Pattern Matching

Yes if the pattern exists in the string, no otherwise. Namely, this automaton:

<img src="img/patternmatching.png" alt="pattern matching automaton" width="300">

In [ ]:
def pattern_matching(P, is_neg=False, max_length = 24_000):
    '''
    Automaton generating strings for the pattern matching problem:
    L = {w \in {a, b, c}* : \exists x, y \in {a, b, c}* (w = x abcabb y)}.
    '''
    # Transition matrix for the automaton
    # We need a notation like this because otherwise it'd get super hard to read/write.
    # T[i][j] means what you write on the tape when in Si and transitioning to Sj
    # When there's a comma, random choices are needed.
    # S0, # S1, # S2, # S3, # S4, # S5, # S6, # Sf
    T = [["b,c", "a",  None, None, None, None, None],   # S0
         ["c",   "a",  "b",  None, None, None, None],   # S1
         ["b",   "a",  None, "c",  None, None, None],    # S2
         ["b,c", None, None, None, "a",  None, None],   # S3
         ["c",   "a",  None, None, None, "b",  None],   # S4
         [None,  "a",  None, "c",  None, None, "b"],    # S5
         [None,  None, None, None, None, None, "a,b,c"]]# S6
    if is_neg:
        T = [T[6], T[0], T[1], T[2], T[3], T[4], T[5]]

    state = "S0"
    end_states = ["S0"] if is_neg else ["S6"]
    tape = "" # Start with the empty tape 
    steps = 0

    # Transitions _to_ the state in question. 
    # - weights are handled by P
    arr = [0, 1, 2, 3, 4, 5, 6, "f"] 

    while True:
        ix = int(state[-1])
        x = random.choices(arr, weights=P[ix], k=1)[0]
        if x == "f":
            break

        next_state = "S" + str(x)
        write_to = T[ix][x]
        if "," in write_to:
            write_to = random.choice(write_to.split(","))
        tape += write_to
        state = next_state
        
        steps += 1
        if max_length is not None:
            if steps >= max_length:
                return None
    if tape == "":
        return None
    return tape

def tape_is_valid(tape, label, repeated):
    if tape is None:
        return False
    if len(tape) < 8:
        return False
    if tape in repeated:
        return False
    if label == 1 and "abcabb" not in tape:
        return False
    if label == 0 and "abcabb" in tape:
        return False
    return True

In [ ]:
#     S0, # S1, # S2, # S3, # S4, # S5, # S6, # Sf
P = [[0.20, 0.80, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],   #S0
     [0.10, 0.10, 0.80, 0.00, 0.00, 0.00, 0.00, 0.00],   #S1
     [0.10, 0.15, 0.00, 0.75, 0.00, 0.00, 0.00, 0.00],   #S2
     [0.25, 0.00, 0.00, 0.00, 0.75, 0.00, 0.00, 0.00],   #S3
     [0.10, 0.20, 0.00, 0.00, 0.00, 0.70, 0.00, 0.00],   #S4
     [0.00, 0.15, 0.00, 0.10, 0.00, 0.00, 0.75, 0.00],   #S5
     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.80, 0.20]]   #S6

P_neg = [P[6], P[0], P[1], P[2], P[3], P[4], P[5]]

corpus = []
repeated = {}

j = 0
pos, neg = 0, 0
pbar = tqdm(total = MAX_DATA_LEN)

while len(corpus) < MAX_DATA_LEN:

    # Positive
    tape = pattern_matching(P)
    if not tape_is_valid(tape, 1, repeated):
        continue
    repeated[tape] = 1
    mislabel = mislabel_point()
    if pos < MAX_DATA_LEN//2:
        pos += 1
        corpus.append({"Entry": tape, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)

    # Negative
    tape = pattern_matching(P_neg, is_neg=True)
    if not tape_is_valid(tape, 0, repeated):
        continue
    repeated[tape] = 1
    mislabel = mislabel_point()
    if neg < MAX_DATA_LEN//2:
        neg += 1
        corpus.append({"Entry": tape, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)


pbar.close()
random.shuffle(corpus)
train, test = corpus[:TRAIN_LEN], corpus[TRAIN_LEN:]

print(len(train), get_label_balance(train), get_average_length(train))
print("---")
print(len(test), get_label_balance(test), get_average_length(test))# We only need 2k on each

with open("Pattern_Matching/corpus_pattern_matching_id_4k_new.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in train]
with open("Pattern_Matching/corpus_pattern_matching_id_4k_new_test.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in test]

In [ ]:
corpora_ood = {}

Q = lambda d: [[0.20 - d/6, 0.80 + d/6, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],                 #S0
               [0.10 - d/12, 0.10 - d/12, 0.80 + d/6, 0.00, 0.00, 0.00, 0.00, 0.00],         #S1
               [0.10 + d*(1/6), 0.15 + d*(1/6), 0.00, 0.75 - d/3, 0.00, 0.00, 0.00, 0.00],   #S2
               [0.25 - d/5, 0.00, 0.00, 0.00, 0.75 + d/5, 0.00, 0.00, 0.00],                 #S3
               [0.10 + d/8, 0.20 + d/8, 0.00, 0.00, 0.00, 0.70 - d/4, 0.00, 0.00],           #S4
               [0.00, 0.15 + d*(1/6), 0.00, 0.10 + d*(1/6), 0.00, 0.00, 0.75 - d/3, 0.00],   #S5
               [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.80 + d/6, 0.20 - d/6]]                 #S6

Q_neg = lambda e: [Q(e)[-1]] + Q(e)[:-1]

for d in tqdm(DELTAS):

    corpus = []
    repeated = {}
    j = 0
    pos, neg = 0, 0

    while len(corpus) < TRAIN_LEN: # We use train to have some slack
        # Positive
        tape = pattern_matching(Q(d))
        if not tape_is_valid(tape, 1, repeated):
            continue
        repeated[tape] = 1
        mislabel = mislabel_point()
        if pos < TRAIN_LEN//2:
            pos += 1
            corpus.append({"Entry": tape, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
            j += 1
    
        # Negative
        tape = pattern_matching(Q_neg(d), is_neg=True)
        if not tape_is_valid(tape, 0, repeated):
            continue
        repeated[tape] = 1
        mislabel = mislabel_point()
        if neg < TRAIN_LEN//2:
            neg += 1
            corpus.append({"Entry": tape, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
            j += 1

    random.shuffle(corpus)
    corpora_ood["delta_{}".format(d)] = corpus
    print(d, len(corpus), get_label_balance(corpus), get_average_length(corpus))

with open("Pattern_Matching/corpus_pattern_matching_ood_4k_new_test.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(corpora_ood, ensure_ascii=False))

## Vending Machine

Two problems:

**Vending machine**: Given a sequence of additions + substractions (selections), and a number (the remaining balance), verify if the sequence leads to the number given.

**Vending machine (with sum)**: Given a sequence of additions and substractions, return the remaining balance. Unlike other problems, this one is not a classification problem.

Namely, this automaton:

<img src="img/vendingmachine.png" alt="Vending machine automaton" width="300px">

In [ ]:
def vending_machine(P, is_neg=False, max_length = 24_000, debug=False):
    """
    Automaton generating strings for the vending machine.
    One coffee costs 15. One soda costs 25. One biscuit costs 20.
    You have 5 and 10 coins.
    This ends with a sequence of items and change (press "change" to end)
    This vending machine continues the transactions until someone clicks "end".
    """
    # Transition matrix for the automaton
    # We need a notation like this because otherwise it'd get super hard to read/write.
    # T[i][j] means what you write on the tape when in Si and transitioning to Sj
    # When there's a comma, random choices are needed.
    #     S0=0, #S1=10, #S2=20,      #S3=5, #S4=15 #S5=25 #Sf=Change
    T = [[None, "+10", None,        "+5", None, None,      "change"], # S0 = 0
         [None, None, "+10",        None, "+5", None,      "change"], # S1 = 10
         ["biscuit", None, "coffee", None, None, "+5",      "change"], # S2 = 20
         [None, None, None,         None, "+10", None,     "change"], # S3 = 5
         ["coffee", None,  None,    None, None, "+10",     "change"], # S4 = 15
         ["soda", "coffee", None,   "biscuit", None, None,  "change"], # S5 = 25
         [None, None, None,         None, None, None,      "change"], # Sf = change
        ]

    state = "S0"
    end_states = ["S0"] if is_neg else ["S6"]
    tape = "" # Start with the empty tape 
    steps = 0

    # Transitions _to_ the state in question. 
    # - weights are handled by P
    arr = [0, 1, 2, 3, 4, 5, "f"] 
    terminate = False

    while True:
        ix = int(state[-1])
        if not terminate:
            x = random.choices(arr, weights=P[ix], k=1)[0]
        else:
            x = "f"
        if x == "f":
            break

        next_state = "S" + str(x)
        if debug:
            print(f"from S{ix} to {next_state}")
        write_to = T[ix][x]
        if "," in write_to:
            write_to = random.choice(write_to.split(","))
        tape += write_to + ","
        if debug:
            print(tape)
        state = next_state
        
        steps += 1
        if max_length is not None:
            if steps >= max_length:
                terminate = True
    if tape == "":
        return None
    return tape

In [ ]:
P = [[0.00, 0.75, 0.00,   0.20, 0.00, 0.00, 0.05], # S0 = 0
     [0.00, 0.00, 0.75,   0.00, 0.20, 0.00, 0.05], # S1 = 10
     [0.40, 0.00, 0.40,   0.00, 0.00, 0.15, 0.05], # S2 = 20
     [0.00, 0.00, 0.00,   0.00, 0.80, 0.00, 0.20], # S3 = 5
     [0.75, 0.00, 0.00,   0.00, 0.00, 0.20, 0.05], # S4 = 15
     [0.40, 0.40, 0.00,   0.15, 0.00, 0.00, 0.05], # S5 = 25
     [0.00, 0.00, 0.00,   0.00, 0.00, 0.00, 1.00], # Sf = change
    ]

def assert_string(string):
    vals = {"biscuit": 20,
            "coffee": 15,
            "soda": 25}
    valid = True
    balances = [0]
    balance = 0
    for e in string.split(","):
        if e == "":
            continue
        if e[0] == "+":
            balance += int(e[1:])
            balances.append(balance)
        else:
            if balance - vals[e] < 0:
                return random.choice(balances), False
            else:
                balance = balance - vals[e]
    return balance, True

In [ ]:
corpus = []
repeated = {}

j = 0
pos, neg = 0, 0

while len(corpus) < MAX_DATA_LEN:
    tape = vending_machine(P)
    if tape is None:
        continue

    if len(tape.split(",")) < 8:
        continue
    
    final_balance, is_valid = assert_string(tape)
    tape += str(final_balance)

    if tape in repeated:
        continue
    repeated[tape] = 1

    mislabel = mislabel_point()
    
    if is_valid and pos < MAX_DATA_LEN//2:
        pos += 1
        corpus.append({"Entry": tape, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
        j += 1
    else:
        if neg < MAX_DATA_LEN//2:
            neg += 1
            corpus.append({"Entry": tape, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
            j += 1

random.shuffle(corpus)
train, test = corpus[:TRAIN_LEN], corpus[TRAIN_LEN:]

print(len(train), get_label_balance(train))
print("---")
print(len(test), get_label_balance(test)) # We only need 2k on each

with open("Vending_Machine/corpus_vending_machine_id_4k.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in train]
with open("Vending_Machine/corpus_vending_machine_id_4k_test.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in test]

In [ ]:
corpora_ood = {}
Q = lambda d: [[0.00, 0.75 + -d/2, 0.00,  0.20 + d/2, 0.00, 0.00, 0.05], # S0 = 0
     [0.00, 0.00, 0.75 - d/2,             0.00, 0.20 + d/2, 0.00, 0.05], # S1 = 10
     [0.40 + (d/16), 0.00, 0.40 + (d/16), 0.00, 0.00, 0.15 - (d/8), 0.05], # S2 = 20
     [0.00, 0.00, 0.00,                   0.00, 0.80 + d/6, 0.00, 0.20 - d/6], # S3 = 5
     [0.75 - d/2, 0.00, 0.00,             0.00, 0.00, 0.20 + d/2, 0.05], # S4 = 15
     [0.40 + (d/16), 0.40 + (d/16), 0.00, 0.15 - (d/8), 0.00, 0.00, 0.05], # S5 = 25
     [0.00, 0.00, 0.00,                   0.00, 0.00, 0.00, 1.00], # Sf = change
    ]

for d in tqdm(DELTAS):

    corpus = []
    repeated = {}
    j = 0
    pos, neg = 0, 0

    while len(corpus) < TRAIN_LEN: # We use train to have some slack
        tape = vending_machine(Q(d))
        if tape is None:
            continue
    
        if len(tape.split(",")) < 8:
            continue
        
        final_balance, is_valid = assert_string(tape)
        tape += str(final_balance)
    
        if tape in repeated:
            continue
        repeated[tape] = 1

        mislabel = mislabel_point()
        
        if is_valid and pos < TRAIN_LEN//2:
            pos += 1
            corpus.append({"Entry": tape, "Label": 1 if not mislabel else 0, "IsOOd": True, "delta": d, "Index": j})
            j += 1
        else:
            if neg < TRAIN_LEN//2:
                neg += 1
                corpus.append({"Entry": tape, "Label": 0 if not mislabel else 1, "IsOOd": True, "delta": d, "Index": j})
                j += 1

    random.shuffle(corpus)
    corpora_ood["delta_{}".format(d)] = corpus
    print(d, len(corpus), get_label_balance(corpus), get_average_length(corpus))

with open("Vending_Machine/corpus_vending_machine_ood_4k_test.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(corpora_ood, ensure_ascii=False))


## Hamiltonian

Given a path and a graph, is the path a Hamiltonian cycle?

In [ ]:
import multiprocessing.pool
def print_graph_statistics(corpus, available_vertices):
    vertexes = {k: 0 for k in available_vertices}
    for e in corpus:
        vertexes[len(e["Entry"])] += 1

    r = lambda x: round(x*100/len(corpus), 1)
    print(k for k in available_vertices)
    print([f"{r(v)}% |" for k, v in vertexes.items()])


# Fixed for eventual upload:
def is_acyclic(graph):
    in_degree = [0 for _ in graph.keys()]
    q = []
    for u, edges in graph.items():
        for v in edges:
            in_degree[int(v)] += 1
    for u in graph.keys():
        if in_degree[int(u)] == 0:
            q.append(u)
    # BFS traversal
    visited = 0
    while q:
        u = q.pop(0)
        visited += 1
        for v in graph[u]:
            in_degree[int(v)] -= 1
            if in_degree[int(v)] == 0:
                q.append(v)

    return visited == len(graph.keys())


def random_graph(n_vertices, weights=None, fully_connected=False, acyclic=True):
    vertices = {str(i): [] for i in range(n_vertices)}

    for u in vertices.keys():
        if acyclic:
            target_sources = [v for v in vertices.keys() if u != v and u not in vertices[v]]
        else:
            target_sources = [v for v in vertices.keys() if u != v]            
        if not fully_connected:
            if weights is None:
                target_sources = [v for v in target_sources if random.randint(0, 1) == 1]
            else:
                target_sources = [v for v in target_sources if random.uniform(0, 1) < weights[int(v)]]
        vertices[u] = target_sources

    if not is_acyclic(vertices):
        if acyclic:
            return None
    
    return vertices


def find_hamiltonian_cycle(graph):

    def is_safe_to_add(v, pos, path): 
        if str(v) not in graph[str(path[pos-1])]: 
            return False
        return v not in path # if v in path return false

    def hamiltonian_cycle_util(path, pos): 
        if pos == len(graph): 
            if str(path[0]) in graph[str(path[pos-1])][path[0]]: 
                return True
            else:
                return False
        for v in range(1, len(graph)): 
            if is_safe_to_add(v, pos, path): 
                path[pos] = v 
                if hamiltonian_cycle_util(path, pos+1):
                    return True
                path[pos] = -1
        return False

    path = [-1 for _ in graph.keys()]
    path[0] = 0
    if not hamiltonian_cycle_util(path, 1): 
        return None
    return [{"Index": i, "Vertex": p} for i, p in enumerate(path)]


In [ ]:
# For ID all weights are uniform, so weights = None is the same as saying 
# weights = [1, 1, 1, 1, ...]
corpus = []
repeated = {}
available_vertices = [5, 7, 10, 12, 15, 20, 25, 30]

Qv = {5: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2],
      7: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                    0.5 + d/16, 0.75 - d/16],
      10: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16],
      12: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2, 1 - d/16],
      15: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2, 1 - d/16, 0.50 + d/16, 0.90 - d/2, 0.6 + d/8],
      20: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2, 1 - d/16, 0.50 + d/16, 0.90 - d/2, 0.6 + d/8,
                     0.50 + d/16, 0.90 - d/2,  0.75 - d/2,  0.6 + d/8,   1 - d/16,],
      25: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2, 1 - d/16, 0.50 + d/16, 0.90 - d/2, 0.6 + d/8,
                     0.50 + d/16, 0.90 - d/2,  0.75 - d/2,  0.6 + d/8,   1 - d/16,
                     0.6 + d/8,   0.50 + d/16, 1 - d/16,    0.75 - d/2, 0.90 - d/2],
      30: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2,  1 - d/16,    0.50 + d/16, 0.90 - d/2,  0.6 + d/8,
                     0.50 + d/16, 0.90 - d/2,  0.75 - d/2,  0.6 + d/8,   1 - d/16,
                     0.6 + d/8,   0.50 + d/16, 1 - d/16,    0.75 - d/2, 0.90 - d/2,
                     0.90 - d/2,  0.6 + d/8,   0.40 + d/2,  1 - d/16, 0.50 + d/16],
     }


j = 0
pos, neg = 0, 0

pbar = tqdm(total=MAX_DATA_LEN)

while len(corpus) < MAX_DATA_LEN:

    vertices = random.choice(available_vertices)
    graph, _ = random_graph(vertices, weights=Qv[vertices](0),
                            fully_connected=False, acyclic=False, print_graph=False)
    tape = graph
    if tape is None:
        continue

    if str(tape) in repeated:
        continue
    repeated[str(tape)] = 1

    mislabel = mislabel_point()
    path = None
    try:
        with multiprocessing.pool.ThreadPool() as pool:
            path = pool.apply_async(find_hamiltonian_cycle, (tape,)).get(timeout=3)
    except:
        continue

    is_valid = path is not None #result == sat

    if is_valid:
        if pos < MAX_DATA_LEN//2:
            pos += 1
            corpus.append({"Entry": tape, "Label": 1 if not mislabel else 0, "IsOOd": False, 
                           "Vertices": vertices, "Path": path, "Index": j})
            j += 1
            pbar.update(1)
    else:
        if neg < MAX_DATA_LEN//2:
            neg += 1
            corpus.append({"Entry": tape, "Label": 0 if not mislabel else 1, "IsOOd": False, 
                           "Vertices": vertices, "Path": path, "Index": j})
            j += 1
            pbar.update(1)

pbar.close()
random.shuffle(corpus)
train, test = corpus[:TRAIN_LEN], corpus[TRAIN_LEN:]

print(len(train), get_label_balance(train))
print("---")
print(len(test), get_label_balance(test)) # We only need 2k on each
print("---")
print_graph_statistics(corpus, available_vertices)

with open("Hamiltonian/corpus_hamiltonian_id_4k.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in train]
with open("Hamiltonian/corpus_hamiltonian_id_4k_test.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in test]
#4219

In [ ]:
corpora_ood = {}

Qv = {5: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2],
      7: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                    0.5 + d/16, 0.75 - d/16],
      10: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16],
      12: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2, 1 - d/16],
      15: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2, 1 - d/16, 0.50 + d/16, 0.90 - d/2, 0.6 + d/8],
      20: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2, 1 - d/16, 0.50 + d/16, 0.90 - d/2, 0.6 + d/8,
                     0.50 + d/16, 0.90 - d/2,  0.75 - d/2,  0.6 + d/8,   1 - d/16,],
      25: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2, 1 - d/16, 0.50 + d/16, 0.90 - d/2, 0.6 + d/8,
                     0.50 + d/16, 0.90 - d/2,  0.75 - d/2,  0.6 + d/8,   1 - d/16,
                     0.6 + d/8,   0.50 + d/16, 1 - d/16,    0.75 - d/2, 0.90 - d/2],
      30: lambda d: [1 - d/16, 0.75 - d/2, 0.40 + d/2, 0.50 + d/16, 0.5 + d/2,
                     0.5 + d/16, 0.75 - d/16, 0.40 + d/2, 0.50 + d/16, 1 - d/16,
                     0.75 - d/2,  1 - d/16,    0.50 + d/16, 0.90 - d/2,  0.6 + d/8,
                     0.50 + d/16, 0.90 - d/2,  0.75 - d/2,  0.6 + d/8,   1 - d/16,
                     0.6 + d/8,   0.50 + d/16, 1 - d/16,    0.75 - d/2, 0.90 - d/2,
                     0.90 - d/2,  0.6 + d/8,   0.40 + d/2,  1 - d/16, 0.50 + d/16],
     }

# They tend to get stuck: just restart when needed
for d in tqdm(DELTAS):

    corpus = []
    repeated = {}
    available_vertices = [5, 7, 10, 12, 15, 20, 25, 30]
    j = 0
    pos, neg = 0, 0
    pbar = tqdm(total=MAX_DATA_LEN)
    while len(corpus) < TRAIN_LEN: # We use train to have some slack
        vertices = random.choice(available_vertices)
        graph, _ = random_graph(vertices, weights=Qv[vertices](d),
                                fully_connected=False, acyclic=False, print_graph=False)
        tape = graph
        if tape is None:
            continue
    
        if str(tape) in repeated:
            continue
        repeated[str(tape)] = 1
    
        mislabel = mislabel_point()
        path = None
        try:
            with multiprocessing.pool.ThreadPool() as pool:
                path = pool.apply_async(find_hamiltonian_cycle, (tape,)).get(timeout=3)
        except:
            continue
        is_valid = path is not None #result == sat
    
        if is_valid and pos < MAX_DATA_LEN//2:
            pos += 1
            corpus.append({"Entry": tape, "Label": 1 if not mislabel else 0, "IsOOd": True,
                           "Vertices": vertices, "Path": path, "Index": j})
            j += 1
            pbar.update(1)
        if not is_valid and neg < MAX_DATA_LEN//2:
            neg += 1
            corpus.append({"Entry": tape, "Label": 0 if not mislabel else 1, "IsOOd": True,
                           "Vertices": vertices, "Path": path, "Index": j})
            j += 1
            pbar.update(1)
    
    pbar.close()    
    random.shuffle(corpus)
    corpora_ood["delta_{}".format(d)] = corpus
    print(f"d: {d}\nlen(corpus): {len(corpus)}\nlabel_balance: {get_label_balance(corpus)}\navg len: {get_average_length(corpus)}")
    print("---")
    print_graph_statistics(corpus, available_vertices)

    with open("Hamiltonian/corpus_hamiltonian_ood_4k_test.json", "a", encoding="utf-8") as f:
        f.write(json.dumps(corpora_ood, ensure_ascii=False))
# 2936

In [ ]:
corpus_id = [json.loads(l) for l in open("Hamiltonian/corpus_hamiltonian_id_4k.json", "r", encoding="utf-8").readlines()]
corpus_id_test = [json.loads(l) for l in open("Hamiltonian/corpus_hamiltonian_id_4k_test.json", "r", encoding="utf-8").readlines()]

print(len(corpus_id), get_label_balance(corpus_id))
print("---")
print(len(corpus_id_test), get_label_balance(corpus_id_test)) # We only need 2k on each
print("---")


def get_noisy_path(graph):
    verts = [k for k in range(len(graph))][1:]
    ts = lambda x: [str(i) for i in x]
    ttype = random.choice([1,2,3,4,5,6])
    if ttype == 5:
        # Too short
        verts = verts[:-1]
        random.shuffle(verts)
        test_path = "0," + ",".join(ts(verts))
    elif ttype == 4:
        # Too long / vertex not in graph
        verts = verts + [len(graph)]
        random.shuffle(verts)
        test_path = "0," + ",".join(ts(verts))
    elif ttype == 3:
        # Repeated paths
        z = random.choice(verts)
        verts.append(z)
        random.shuffle(verts)
        test_path = "0," + ",".join(ts(verts))
    else:
        # Just plain wrong lol -- we know that this one is not true because 
        # the graphs sent here are either noisy or not Hamiltonian
        random.shuffle(verts)
        test_path = "0," + ",".join(ts(verts))
    return test_path


def get_fake_paths(corpus):
# We need to inject fake paths
    new_corpus_id = []
    for p in corpus:
        _p = deepcopy(p)
        if p["Label"] == 1:
            if _p["PathGraph"] is not None:
                _p["Path"] = ",".join([str(v["Vertex"]) for v in _p["PathGraph"]])
            else:
                # It is one of the noisy entries
                _p["Path"] = get_noisy_path(p["Entry"])
        else:
            _p["Path"] = get_noisy_path(p["Entry"])
        new_corpus_id.append(_p)
    return new_corpus_id

new_corpus_id = get_fake_paths(corpus_id)
new_corpus_id_test = get_fake_paths(corpus_id_test)

print(len(new_corpus_id), get_label_balance(new_corpus_id))
print("---")
print(len(new_corpus_id_test), get_label_balance(new_corpus_id_test)) # We only need 2k on each
print("---")

print(new_corpus_id[0])

In [ ]:
random.shuffle(new_corpus_id)
with open("Hamiltonian/corpus_hamiltonian_id_4k.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in new_corpus_id]
random.shuffle(new_corpus_id_test)
with open("Hamiltonian/corpus_hamiltonian_id_4k_test.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in new_corpus_id_test]

In [ ]:
corpus_od_test_w_negs = [json.loads(l) for l in open("Hamiltonian/corpus_hamiltonian_ood_4k_test.json", "r", encoding="utf-8").readlines()]

In [ ]:
for k, v in corpus_od_test_w_negs[0].items():
    print(k)
    print(len(v), get_label_balance(v))
    print("---")

new_od = [
    {
        k: get_fake_paths(v) for k, v in corpus_od_test_w_negs[0].items()
    }
]

for k, v in new_od[0].items():
    print(k)
    print(len(v), get_label_balance(v))
    print("---")

In [ ]:
# There are wayyyy too many 1s. 20% or so are 0s -- so around 600. So we need to pick 600 1s to get a 50-50 test set.
for k, v in new_od[0].items():
    zeros = [e for e in v if e["Label"] == 0]
    ones = [e for e in v if e["Label"] == 1][:len(zeros)]
    new_v = zeros + ones
    random.shuffle(new_v)
    for i, _ in enumerate(new_v):
        new_v[i]["Index"] = i
    new_od[0][k] = new_v

for k, v in new_od[0].items():
    print(k)
    print(len(v), get_label_balance(v))
    print("---")


In [ ]:
with open("Hamiltonian/corpus_hamiltonian_ood_4k_test.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(new_od[0], ensure_ascii=False))

## Mazes

Two problems: 

**Maze Complete**: Given a maze, some moves, and a cut path, decide if the moves given complete the path (up to 3 moves).

**Maze Solve**: Given a maze and a path, decide if the path leads to a completion (i.e., verify the path).

```
!pip install mazelib
```

### Maze Complete

In [ ]:
# !pip install mazelib
from mazelib import Maze
from mazelib.generate.Ellers import Ellers
from mazelib.solve.ShortestPaths import ShortestPaths
from collections import Counter

In [ ]:
def get_maze(shift, sx=6, sy=6):
    m = Maze()
    # Ensure we get a sort-of-diverse set
    xshift = shift
    yshift = 1 - shift
    if random.choice([0, 1]) == 1:
        xshift = 1 - shift
        yshift = shift
    m.generator = Ellers(sx, sy, xskew=xshift, yskew=yshift) #xskew=0.85, yskew=0.15)
    m.generate()
    m.start = (0, 1)
    m.end = (2*sx - 1, 2*sy)
    #print(m.tostring(True, True))
    #showPNG(m.grid)
    m.solver = ShortestPaths()
    m.solve()
    #m.generate_entrances()
    return m.tostring(True), m.tostring(True, True), m

In [ ]:
def get_neighbours(split_string, i, j):
    """
    Neighbours in a Moore neighbourhood of i,j
    """
    neighbours = []
    for a, b in [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]:
        if split_string[a][b] == "+":
            neighbours.append((a, b))
    return neighbours


def get_path(maze):
    solns = maze.solutions[0]
    previous = solns[0]
    answer = []
    for r in solns[1:]:
        row, column = r
        if row == previous[0]:
            if column == previous[1] - 1:
                answer.append("left")
            else:
                answer.append("right")
        elif row == previous[0] + 1:
            answer.append("down")
        elif row == previous[0] - 1:
            answer.append("up")
        previous = r
    return ",".join(answer)


def add_noise(maze, solved_maze_string, ceilings, debug=False):
    split_string = solved_maze_string.split("\n")
    
    c = Counter()
    solns = maze.solutions[0]
    positions = {r[0]: [] for r in solns}
    for r in solns:
        row, position = r
        c[row] += 1
        positions[row].append(position)
    
    ins = lambda s, i: s[:i] + " " + s[i + 1:]
    ins2 = lambda s, i: s[:i] + "?" + s[i + 1:]

    if debug:
        for i, s in enumerate(split_string):
            print(i, "\t", s)
        print("\n\n")
        print(c)
    
    ix = random.choice([k for k in positions.keys() if k > ceilings[0] and k < ceilings[1]])
    point = random.choice(positions[ix])
    new_split_string = [s for s in split_string]
    answer = ""
    
    neighbours = get_neighbours(split_string, ix, point)
    i, j = random.choice(neighbours)
    if debug: print(f"ix, point: {ix, point} | i, j: {i, j}")

    iy, jy = 0, 0
    if i == ix:
        if j == point - 1:
            new_split_string[i] = ins2(split_string[i], j)
            new_split_string[ix] = ins(new_split_string[ix], point)
            neighbours_2 = get_neighbours(new_split_string, ix, point)
            iy, jy = ix, point
        else: #j == point + 1:
            new_split_string[i] = ins(split_string[i], j)
            new_split_string[ix] = ins2(new_split_string[ix], point)
            neighbours_2 = get_neighbours(new_split_string, i, j)
            iy, jy = i, j
        answer = "right,right"
    elif i == ix - 1: #j == point (moore)
            new_split_string[i] = ins2(split_string[i], j)
            new_split_string[ix] = ins(new_split_string[ix], point)
            neighbours_2 = get_neighbours(new_split_string, ix, point)
            iy, jy = ix, point
            answer = "down,down"
    elif i == ix + 1: #j == point:
            new_split_string[i] = ins(split_string[i], j)
            new_split_string[ix] = ins2(new_split_string[ix], point)
            neighbours_2 = get_neighbours(new_split_string, i, j)
            iy, jy = i, j
            answer = "down,down"

    try:
        i, j  = random.choice(neighbours_2)
    except:
        #print(f"ix, point: {ix, point} | i, j: {i, j} | neighbours_2: {neighbours_2}")
        return None, None, None
    if debug: print(f"k, l: {i, j}")

    if i == iy:
        if j == jy - 1:
            new_split_string[i] = ins(new_split_string[i], j)
            new_split_string[iy] = ins(new_split_string[iy], jy)
        else: #j == point + 1:
            new_split_string[i] = ins(new_split_string[i], j)
            new_split_string[iy] = ins(new_split_string[iy], jy)
        answer += ",right"
    elif i == iy - 1: #j == point (moore)
            new_split_string[i] = ins(new_split_string[i], j)
            new_split_string[iy] = ins(new_split_string[iy], jy)
            answer += ",down"
    elif i == iy + 1: #j == point:
            new_split_string[i] = ins(new_split_string[i], j)
            new_split_string[iy] = ins(new_split_string[iy], jy)
            answer += ",down"

    if debug:
        for i, s in enumerate(new_split_string):
            print(i, "\t", s)

    options = ["right", "down"]
    bad_answer = answer.split(",")
    i = random.choice([i for i in range(len(bad_answer))])
    selection = random.choice([i for i in range(5)])
    if selection == 0: # 20% of the bad_answers will be overly long
        bad_answer.append(random.choice(options))
    elif selection == 1: # 20% of the bad_answers will be short
        bad_answer = bad_answer[:-1]
    else:
        if bad_answer[i] == "right":
            bad_answer[i] = "down"
        else:
            bad_answer[i] = "right"
    
    final_string = "\n".join(new_split_string)
    return final_string, answer, ",".join(bad_answer)


def distro(d):
    # We want to have 50-50 chance in the 0.85
    #if d == 0.85:
    #    return 0.5 #0.85 - 0.4*0.85
    #return 0.85 - 0.4*d
    return 0.5 + 0.45*d

In [ ]:
corpus = []
repeated = {}

available_sizes = [3, 4, 5, 6]
ceilings = {3: [3, 5],
            4: [3, 7],
            5: [3, 9],
            6: [3, 11]}

d = 0
j = 0
pos, neg = 0, 0

pbar = tqdm(total=MAX_DATA_LEN)

while len(corpus) < MAX_DATA_LEN:
    ks = random.choice(available_sizes)
    maze_string, solved_maze_string, maze = get_maze(distro(d), sx=ks, sy=ks)
    final_maze_string, answer, noise_answer = add_noise(maze, solved_maze_string, ceilings=ceilings[ks])
    if answer is None:
        continue
    path = get_path(maze)

    if maze_string in repeated:
        continue
    repeated[maze_string] = 1

    tape_entry = f"Solved maze:\n{final_maze_string}\nMissing moves:\n{answer}"
    bad_tape_entry = f"Solved maze:\n{final_maze_string}\nMissing moves:\n{noise_answer}"

    mislabel = mislabel_point()
    if pos < MAX_DATA_LEN//2:
        pos += 1
        corpus.append({"Entry": tape_entry, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)
    if neg < MAX_DATA_LEN//2:
        neg += 1
        corpus.append({"Entry": bad_tape_entry, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)

pbar.close()
random.shuffle(corpus)
train, test = corpus[:TRAIN_LEN], corpus[TRAIN_LEN:]

print(len(train), get_label_balance(train))
print("---")
print(len(test), get_label_balance(test)) # We only need 2k on each
print("---")

with open("MazeComplete/corpus_maze_complete_id_4k.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in train]
with open("MazeComplete/corpus_maze_complete_id_4k_test.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in test]

In [ ]:
corpora_ood = {}

for d in tqdm(DELTAS):

    corpus = []
    repeated = {}
    pos, neg = 0, 0
    j = 0
    pbar = tqdm(total=TRAIN_LEN)
    while len(corpus) < TRAIN_LEN:

        ks = random.choice(available_sizes)
        maze_string, solved_maze_string, maze = get_maze(distro(d), sx=ks, sy=ks)
        final_maze_string, answer, noise_answer = add_noise(maze, solved_maze_string, ceilings=ceilings[ks])
        if answer is None:
            continue
        path = get_path(maze)
    
        if maze_string in repeated:
            continue
        repeated[maze_string] = 1
    
        tape_entry = f"Solved maze:\n{final_maze_string}\nMissing moves:\n{answer}"
        bad_tape_entry = f"Solved maze:\n{final_maze_string}\nMissing moves:\n{noise_answer}"
    
        mislabel = mislabel_point()
        if pos < MAX_DATA_LEN//2:
            pos += 1
            corpus.append({"Entry": tape_entry, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
            j += 1
            pbar.update(1)
        if neg < MAX_DATA_LEN//2:
            neg += 1
            corpus.append({"Entry": bad_tape_entry, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
            j += 1
            pbar.update(1)


    pbar.close()
    random.shuffle(corpus)
    corpora_ood["delta_{}".format(d)] = corpus
    print("label balance")
    get_label_balance(corpus)
    print("avg_len")
    get_average_length(corpus)
    print(f"d: {d}\nlen(corpus): {len(corpus)}")

with open("MazeComplete/corpus_maze_complete_ood_4k_test.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(corpora_ood, ensure_ascii=False))
# 2936

### Maze Solve

In [ ]:
def get_maze(shift, sx=6, sy=6):
    m = Maze()
    # Ensure we get a sort-of-diverse set
    xshift = shift
    yshift = 1 - shift
    if random.choice([0, 1]) == 1:
        xshift = 1 - shift
        yshift = shift
    m.generator = Ellers(sx, sy, xskew=xshift, yskew=yshift) #xskew=0.85, yskew=0.15)
    m.generate()
    m.start = (0, 1)
    m.end = (2*sx - 1, 2*sy)
    #print(m.tostring(True, True))
    #showPNG(m.grid)
    m.solver = ShortestPaths()
    m.solve()
    #m.generate_entrances()
    return m.tostring(True), m.tostring(True, True), m

In [ ]:
def get_path(maze):
    solns = maze.solutions[0]
    previous = solns[0]
    answer = []
    for r in solns[1:]:
        row, column = r
        if row == previous[0]:
            if column == previous[1] - 1:
                answer.append("left")
            else:
                answer.append("right")
        elif row == previous[0] + 1:
            answer.append("down")
        elif row == previous[0] - 1:
            answer.append("up")
        previous = r

    options = ["right", "down", "up", "down"]
    bad_answer = answer[:]
    selection = random.choice([i for i in range(5)])
    if selection == 0: # 20% of the bad_answers will be overly long
        bad_answer.append(random.choice(options))
    elif selection == 1: # 20% of the bad_answers will be short
        bad_answer = bad_answer[:-1]
    else:
        i = random.choice([i for i in range(len(bad_answer))])
        replacement = random.choice([o for o in options if o != bad_answer[i]])
        bad_answer[i] = replacement
        selection = random.choice([t for t in range(4)])
        if selection < 1:
            i = random.choice([i for i in range(len(bad_answer))])
            replacement = random.choice([o for o in options if o != bad_answer[i]])
            bad_answer[i] = replacement

    return ",".join(answer), ",".join(bad_answer)

def distro(d):
    # We want to have 50-50 chance in the 0.85
    #if d == 0.85:
    #    return 0.5 #0.85 - 0.4*0.85
    #return 0.85 - 0.4*d
    return 0.5 + 0.45*d

In [ ]:
corpus = []
repeated = {}

available_sizes = [3, 4, 6, 8, 10]

d = 0
j = 0
pos, neg = 0, 0

pbar = tqdm(total=MAX_DATA_LEN)

while len(corpus) < MAX_DATA_LEN:
    ks = random.choice(available_sizes)
    maze_string, solved_maze_string, maze = get_maze(distro(d), sx=ks, sy=ks)
    good_path, bad_path = get_path(maze)

    if solved_maze_string in repeated:
        continue
    repeated[solved_maze_string] = 1

    tape_entry = f"Solved maze:\n{solved_maze_string}\nSolution:\n{good_path}"
    bad_tape_entry = f"Solved maze:\n{solved_maze_string}\nSolution:\n{bad_path}"

    mislabel = mislabel_point()
    if pos < MAX_DATA_LEN//2:
        pos += 1
        corpus.append({"Entry": tape_entry, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)
    if neg < MAX_DATA_LEN//2:
        neg += 1
        corpus.append({"Entry": bad_tape_entry, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)

pbar.close()
random.shuffle(corpus)
train, test = corpus[:TRAIN_LEN], corpus[TRAIN_LEN:]

print(len(train), get_label_balance(train))
print("---")
print(len(test), get_label_balance(test)) # We only need 2k on each
print("---")

with open("MazeSolve/corpus_maze_solve_id_4k.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in train]
with open("MazeSolve/corpus_maze_solve_id_4k_test.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in test]

In [ ]:
corpora_ood = {}

for d in tqdm(DELTAS):

    corpus = []
    repeated = {}
    pos, neg = 0, 0
    j = 0
    pbar = tqdm(total=TRAIN_LEN)
    while len(corpus) < TRAIN_LEN:

        ks = random.choice(available_sizes)
        maze_string, solved_maze_string, maze = get_maze(distro(d), sx=ks, sy=ks)
        good_path, bad_path = get_path(maze)
    
        if solved_maze_string in repeated:
            continue
        repeated[solved_maze_string] = 1
    
        tape_entry = f"Solved maze:\n{solved_maze_string}\nSolution:\n{good_path}"
        bad_tape_entry = f"Solved maze:\n{solved_maze_string}\nSolution:\n{bad_path}"
    
        mislabel = mislabel_point()
        if pos < MAX_DATA_LEN//2:
            pos += 1
            corpus.append({"Entry": tape_entry, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
            j += 1
            pbar.update(1)
        if neg < MAX_DATA_LEN//2:
            neg += 1
            corpus.append({"Entry": bad_tape_entry, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
            j += 1
            pbar.update(1)

    pbar.close()
    random.shuffle(corpus)
    corpora_ood["delta_{}".format(d)] = corpus
    print("label balance")
    get_label_balance(corpus)
    print("avg_len")
    get_average_length(corpus)
    print(f"d: {d}\nlen(corpus): {len(corpus)}")

with open("MazeSolve/corpus_maze_solve_ood_4k_test.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(corpora_ood, ensure_ascii=False))
# 2936

## Stack

A standard push-pop-stop stack sequence of operations. In the positive case, it is well-formed; otherwise the results do not lead to the output or violate some of the rules.

In [ ]:
def pushpop(k, min_seq=None, this_tape=None, probs = [0.25, 0.25, 0.25, 0.5]):
    """
    Probs: [ Pr[push], Pr[pop], Pr[stop], Pr[0] if push ]
    So Pr[0] = 1 - Pr[1].
    """
    initial_string = "".join([str(j) for j in random.choices([0, 1], k = k)])
    if this_tape is not None:
        initial_string = this_tape
    _probs = probs[:3]
    state_sequences = []
    is_stop = False
    count = 0
    while not is_stop:
        next_state = random.choices(["push", "pop", "stop"], weights=_probs, k=1)[0]
        count += 1
        if next_state != "stop":
            state_sequences.append(next_state)
            if next_state == "push":
                state_sequences.append(random.choices(["0", "1"], 
                                                      weights=[probs[-1], 1 - probs[-1]], 
                                                      k=1)[0])
        else:
            if min_seq is None or min_seq <= count:
                is_stop = True
                state_sequences.append(next_state)
                break

    this_string = initial_string[:]
    for i in range(len(state_sequences)):
        state = state_sequences[i]
        if state == "pop":
            if this_string == "":
                continue
            if len(this_string) == 1:
                this_string = ""
            else:
                this_string = this_string[:-1]
        if state == "push":
            continue
        if state in ["0", "1"]:
            this_string = this_string + state
        if state == "stop":
            break

    if this_string == "":
        this_string = "empty"
    return initial_string, state_sequences, this_string


In [ ]:
corpus = []
repeated = {}

available_starting_lengths = [3, 5, 10, 15, 20, 25, 30]
min_seq_lengths = [5, 10, 15]

Q = {0: [0.4, 0.4, 0.20, 0.5]}

j = 0
pos, neg = 0, 0

pbar = tqdm(total=MAX_DATA_LEN)

while len(corpus) < MAX_DATA_LEN:
    ks = random.choice(available_starting_lengths)
    min_seq_len = random.choice(min_seq_lengths)
    initial_string, sequences_list, final_state = pushpop(ks, min_seq = min_seq_len, probs = Q[0])
    tape = f"{initial_string}"
    sequences = ','.join(sequences_list)
    tape_entry = f"{initial_string}\n{sequences}\n{final_state}"
    repeated[tape] = {sequences: final_state}

    for i in range(5):
        _initial_string, _sequences_list, _final_state = pushpop(ks, this_tape=tape, min_seq = min_seq_len, probs = Q[0])
        _sequences = ','.join(_sequences_list)
        repeated[tape][_sequences] = _final_state

    bad_state = None
    if len(repeated[tape]) > 1:
        bad_sequence = random.choice([k for k in repeated[tape] if k != sequences])
        bad_state = repeated[tape][bad_sequence]
        if bad_state == final_state:
            bad_state = None
        else:
            bad_tape_entry = f"{initial_string}\n{bad_sequence}\n{final_state}"

    mislabel = mislabel_point()
    if pos < MAX_DATA_LEN//2:
        pos += 1
        corpus.append({"Entry": tape_entry, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)
    if bad_state is not None:
        if neg < MAX_DATA_LEN//2:
            neg += 1
            corpus.append({"Entry": bad_tape_entry, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
            j += 1
            pbar.update(1)

pbar.close()
random.shuffle(corpus)
train, test = corpus[:TRAIN_LEN], corpus[TRAIN_LEN:]

print(len(train), get_label_balance(train))
print("---")
print(len(test), get_label_balance(test)) # We only need 2k on each
print("---")

with open("Stack/corpus_stack_id_4k.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in train]
with open("Stack/corpus_stack_id_4k_test.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in test]

In [ ]:
corpora_ood = {}

Q = {"0": [0.4, 0.4, 0.20, 0.5],
    "0.2":  [0.4 + 0.2/4, 0.4 + 0.2/4, 0.20 - 0.2/2, 0.5 + 0.2/4],
    "0.45": [0.4 + 0.45/2, 0.4 - 0.45/8, 0.20 - 0.45/4, 0.5 + 0.45/4],
    "0.65": [0.4 + 0.65/2.5, 0.4 - 0.65/5, 0.20 - 0.65/5, 0.5 + 0.65/4],
    "0.85": [0.4 + 0.85/3, 0.4 - 0.85/6, 0.20 - 0.85/(6), 0.5 + 0.85/4]
    }


for d in tqdm(DELTAS):

    corpus = []
    repeated = {}
    pos, neg = 0, 0
    pbar = tqdm(total=TRAIN_LEN)
    while len(corpus) < TRAIN_LEN:
        ks = random.choice(available_starting_lengths)
        min_seq_len = random.choice(min_seq_lengths)
        initial_string, sequences_list, final_state = pushpop(ks, min_seq = min_seq_len, probs = Q[str(d)])
        tape = f"{initial_string}"
        sequences = ','.join(sequences_list)
        tape_entry = f"{initial_string}\n{sequences_list}\n{final_state}"
        repeated[tape] = {sequences: final_state}
    
        for i in range(5):
            _initial_string, _sequences_list, _final_state = pushpop(ks, this_tape=tape, min_seq = min_seq_len, probs = Q[str(d)])
            _sequences = ','.join(_sequences_list)
            repeated[tape][_sequences] = _final_state
    
        bad_state = None
        if len(repeated[tape]) > 1:
            bad_sequence = random.choice([k for k in repeated[tape] if k != sequences])
            bad_state = repeated[tape][bad_sequence]
            if bad_state == final_state:
                bad_state = None
            else:
                bad_tape_entry = f"{initial_string}\n{bad_sequence}\n{final_state}"
    
        mislabel = mislabel_point()
        if pos < TRAIN_LEN//2:
            pos += 1
            corpus.append({"Entry": tape_entry, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
            j += 1
            pbar.update(1)
        if bad_state is not None:
            if neg < TRAIN_LEN//2:
                neg += 1
                corpus.append({"Entry": bad_tape_entry, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
                j += 1
                pbar.update(1)

    pbar.close()
    random.shuffle(corpus)
    corpora_ood["delta_{}".format(d)] = corpus
    print("label balance")
    get_label_balance(corpus)
    print("avg_len")
    get_average_length(corpus)
    print(f"d: {d}\nlen(corpus): {len(corpus)}")

with open("Stack/corpus_stack_ood_4k_test.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(corpora_ood, ensure_ascii=False))
# 2936

# Reversal

Or "is this string up to a marker its own palindrome"?

Example:

abcad#dacba = yes

abcad#ddddd = no

In [3]:
ALPHABET = ["gfx", "chtte", "%", "ltintprk", "¯\\_(ツ)_/¯"]
MIN_LEN = 6

def reversal_tape(P):
    # n + 1 states: ALPHABET + "final"
    tape = []
    end_state = "stop"
    current_state = "start"
    states = [a for a in ALPHABET] + [end_state]
    while True:
        next_state = random.choices(states, weights=P[current_state])[0]
        if next_state == end_state:
            break
        else:
            tape.append(next_state)
        current_state = next_state

    return tape

def add_noise(arr, max_num_noise=3):
    _arr = [a for a in arr]
    for i in range(max_num_noise):
        noise_type = random.choice([1,2,3])
        if noise_type == 1:
            ix = random.choice([i for i in range(len(_arr))])
            _ = _arr.pop(ix)
        elif noise_type == 2:
            ix = random.choice([i for i in range(len(_arr))])
            el = random.choice(ALPHABET)
            _arr = _arr[:ix] + [el] + _arr[ix + 1:]
        elif noise_type == 3:
            _arr = _arr[::-1]

        should_break = random.choice([1,2,3])
        if should_break < 2:
            break
    return _arr

In [ ]:
P = {"start": [0.2, 0.2, 0.2, 0.2, 0.2, 0.],
     "gfx":        [0.30, 0.10, 0.10, 0.10, 0.10, 0.3],
     "chtte":      [0.10, 0.30, 0.10, 0.10, 0.10, 0.3],
     "%":          [0.10, 0.10, 0.30, 0.10, 0.10, 0.3],
     "ltintprk":   [0.10, 0.10, 0.10, 0.30, 0.10, 0.3],
     "¯\\_(ツ)_/¯":[0.10, 0.10, 0.10, 0.10, 0.30, 0.3]
    }

corpus = []
repeated = {}

j = 0
pos, neg = 0, 0
pbar = tqdm(total=MAX_DATA_LEN)

while len(corpus) < MAX_DATA_LEN:

    tape = reversal_tape(P)
    if len(tape) < MIN_LEN:
        continue
    reversed_tape = add_noise(tape)

    tape_entry = f"{''.join(tape)}#{''.join(tape[::-1])}"
    bad_tape_entry = f"{''.join(tape)}#{''.join(reversed_tape)}"

    mislabel = mislabel_point()
    if pos < MAX_DATA_LEN//2:
        pos += 1
        corpus.append({"Entry": tape_entry, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
        j += 1
        pbar.update(1)
    if bad_state is not None:
        if neg < MAX_DATA_LEN//2:
            neg += 1
            corpus.append({"Entry": bad_tape_entry, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
            j += 1
            pbar.update(1)

pbar.close()
random.shuffle(corpus)
print("avg_len")
get_average_length(corpus)
train, test = corpus[:TRAIN_LEN], corpus[TRAIN_LEN:]
print(len(train), get_label_balance(train))
print("---")
print(len(test), get_label_balance(test)) # We only need 2k on each
print("---")


with open("Reversal/corpus_reversal_id_4k.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in train]
with open("Reversal/corpus_reversal_id_4k_test.json", "w", encoding="utf-8") as f:
    _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in test]

In [ ]:
corpora_ood = {}

Q = {"0": {"start": [0.2, 0.2, 0.2, 0.2, 0.2, 0.],
           "gfx":        [0.30, 0.15, 0.15, 0.15, 0.15, 0.1],
           "chtte":      [0.15, 0.30, 0.15, 0.15, 0.15, 0.1],
           "%":          [0.15, 0.15, 0.30, 0.15, 0.15, 0.1],
           "ltintprk":   [0.15, 0.15, 0.15, 0.30, 0.15, 0.1],
           "¯\\_(ツ)_/¯":[0.15, 0.15, 0.15, 0.15, 0.30, 0.1]
           },
    "0.2":  {"start": [0.2, 0.2, 0.2, 0.2, 0.2, 0.],
           "gfx":        [0.25, 0.16, 0.16, 0.16, 0.16, 0.08],
           "chtte":      [0.16, 0.25, 0.16, 0.16, 0.16, 0.08],
           "%":          [0.16, 0.16, 0.25, 0.16, 0.16, 0.08],
           "ltintprk":   [0.16, 0.16, 0.16, 0.25, 0.16, 0.08],
           "¯\\_(ツ)_/¯":[0.16, 0.16, 0.16, 0.16, 0.25, 0.08]
           },
    "0.45": {"start": [0.2, 0.2, 0.2, 0.2, 0.2, 0.],
           "gfx":        [0.22, 0.18, 0.18, 0.18, 0.18, 0.06],
           "chtte":      [0.18, 0.22, 0.18, 0.18, 0.18, 0.06],
           "%":          [0.18, 0.18, 0.22, 0.18, 0.18, 0.06],
           "ltintprk":   [0.18, 0.18, 0.18, 0.22, 0.18, 0.06],
           "¯\\_(ツ)_/¯":[0.18, 0.18, 0.18, 0.18, 0.22, 0.06]
           },
    "0.65": {"start": [0.2, 0.2, 0.2, 0.2, 0.2, 0.],
           "gfx":        [0.20, 0.19, 0.19, 0.19, 0.19, 0.04],
           "chtte":      [0.19, 0.20, 0.19, 0.19, 0.19, 0.04],
           "%":          [0.19, 0.19, 0.20, 0.19, 0.19, 0.04],
           "ltintprk":   [0.19, 0.19, 0.19, 0.20, 0.19, 0.04],
           "¯\\_(ツ)_/¯":[0.19, 0.19, 0.19, 0.19, 0.20, 0.04]
           },
    "0.85": {"start": [0.2, 0.2, 0.2, 0.2, 0.2, 0.],
           "gfx":        [0.19, 0.19, 0.19, 0.19, 0.19, 0.02],
           "chtte":      [0.19, 0.19, 0.19, 0.19, 0.19, 0.02],
           "%":          [0.19, 0.19, 0.19, 0.19, 0.19, 0.02],
           "ltintprk":   [0.19, 0.19, 0.19, 0.19, 0.19, 0.02],
           "¯\\_(ツ)_/¯":[0.19, 0.19, 0.19, 0.19, 0.19, 0.02]
           },
    }

for d in tqdm(DELTAS):

    corpus = []
    repeated = {}
    pos, neg = 0, 0
    pbar = tqdm(total=TRAIN_LEN)
    while len(corpus) < TRAIN_LEN:
        tape = reversal_tape(Q[str(d)])
        if len(tape) < MIN_LEN:
            continue
        reversed_tape = add_noise(tape)
    
        tape_entry = f"{''.join(tape)}#{''.join(tape[::-1])}"
        bad_tape_entry = f"{''.join(tape)}#{''.join(reversed_tape)}"

        mislabel = mislabel_point()
        if pos < TRAIN_LEN//2:
            pos += 1
            corpus.append({"Entry": tape_entry, "Label": 1 if not mislabel else 0, "IsOOd": False, "Index": j})
            j += 1
            pbar.update(1)
        if bad_state is not None:
            if neg < TRAIN_LEN//2:
                neg += 1
                corpus.append({"Entry": bad_tape_entry, "Label": 0 if not mislabel else 1, "IsOOd": False, "Index": j})
                j += 1
                pbar.update(1)

    pbar.close()
    random.shuffle(corpus)
    corpora_ood["delta_{}".format(d)] = corpus
    print("label balance")
    get_label_balance(corpus)
    print("avg_len")
    get_average_length(corpus)
    print(f"d: {d}\nlen(corpus): {len(corpus)}")

with open("Reversal/corpus_reversal_ood_4k_test.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(corpora_ood, ensure_ascii=False))
# 2936

# Alternate versions

Namely, evaluate whether a learner can realise alternative concept classes

In [22]:
import json
import random

problems = {
    "PARITY": ["corpus_parity_id_4k_new.json", "corpus_parity_id_4k_new_test.json", "corpus_parity_ood_4k_test_new_with_negs.json"],
    "Pattern_Matching": ["corpus_pattern_matching_id_4k_new.json", "corpus_pattern_matching_id_4k_new_test.json", "corpus_pattern_matching_ood_4k_new_test.json"],
    "Vending_Machine": ["corpus_vending_machine_id_4k.json", "corpus_vending_machine_id_4k_test.json", "corpus_vending_machine_ood_4k_test.json"],
    "Hamiltonian": ["corpus_hamiltonian_id_4k.json", "corpus_hamiltonian_id_4k_test.json", "corpus_hamiltonian_ood_4k_test.json"],
    "Stack": ["corpus_stack_id_4k.json", "corpus_stack_id_4k_test.json", "corpus_stack_ood_4k_test.json"],
    "MazeComplete": ["corpus_maze_complete_id_4k.json", "corpus_maze_complete_id_4k_test.json", "corpus_maze_complete_ood_4k_test.json"],
    "MazeSolve": ["corpus_maze_solve_id_4k.json", "corpus_maze_solve_id_4k_test.json", "corpus_maze_solve_ood_4k_test.json"],
    "Reversal": ["corpus_reversal_id_4k.json", "corpus_reversal_id_4k_test.json", "corpus_reversal_ood_4k_test.json"]
}


In [33]:
for p, files in problems.items():
    for f in files:
        data = [json.loads(l) for l in open(f"{p}/{f}", "r", encoding="utf-8").readlines()]
        ofile = f.replace(".json", "_shuffled_labels.json")
        if "ood" in f:
            new_dataset = {}
            for delta, delta_dataset in data[0].items():
                new_dataset[delta] = []
                for e in delta_dataset:
                    e["Label"] = random.choice([0, 1])
                    new_dataset[delta].append(e)
            with open(f"{p}/{ofile}", "w", encoding="utf-8") as f:
                f.write(json.dumps(new_dataset, ensure_ascii=False))
        else:
            corpus_shuffled = []
            for e in data:
                e["Label"] = random.choice([0, 1])
                corpus_shuffled.append(e)
            with open(f"{p}/{ofile}", "w", encoding="utf-8") as f:
                _ = [f.write(json.dumps(l, ensure_ascii=False) +"\n") for l in corpus_shuffled]
